In [1]:
from utils.sample import Sample
from scipy.optimize import brentq
import pandas as pd
import numpy as np
import math
import pickle
import scipy.stats as st
import matplotlib.pyplot as plt
from utils.biseccion import bisec
from utils.black_scholes import raiz_ratio, raiz, call_price, d1
from sklearn.metrics import r2_score

In [7]:
i = 0
sampl = []
result = []
while i < 10**4:
    S = np.random.uniform(50, 100)
    K = np.random.uniform(S/2, S*2)
    T = np.random.uniform(0.2, 1.1)
    r = np.random.uniform(0.02, 0.1)
    o = np.random.uniform(0.01, 1)
    c = call_price(S, K, r, o, T)
    if c > 0:
        result.append(o)
        sampl.append([c , S, K, r, T])

        i += 1

In [8]:
df = pd.DataFrame(sampl, columns=['c', 'S', 'k', 'r', 'T'])
df['o'] = result
df.head()

,c,S,k,r,T,o
0,0.000439,70.004001,125.913466,0.039006,0.936373,0.155789
1,22.324878,95.187395,75.949322,0.090688,0.403722,0.264542
2,0.000412,97.709717,174.789619,0.061999,0.624575,0.183804
3,20.417102,72.755482,77.888127,0.032559,1.095930,0.715676
4,31.881417,96.461719,72.737867,0.098655,0.957303,0.334267


In [9]:
# Tiro los casos en que no se puede aplicar el metodo de biseccion
# y calculo la volatilidad implícita
vol_bisec = []
vol_brent = []
i = 0
drops = []
for c, S, k, r, T in sampl:
    f = lambda x: raiz(c, S, k, r, x, T)
    # que se cumpla la precondicción
    if f(0.01) < 0:
        # máxima precision
        o_bic = bisec(f, 0.01, 1, 2**-56)
        o_bren = brentq(f, 0.01, 1, xtol=2**-56)
        vol_bisec.append(o_bic)
        vol_brent.append(o_bren)
    else: 
        drops.append(i)
    i += 1
    
    if i % 10000 == 0:
        print('{}%'.format(100*i/len(sampl)))
    
    
dfb = df.drop(drops)
#volatilidad implicita
dfb['o_bis'] = vol_bisec
dfb['o_bren'] = vol_brent

100.0%


In [10]:
# error entre la volatilidad estimada e implícita
dfb['diff_bis'] = (dfb['o'] - dfb['o_bis']).apply(abs)
dfb['diff_bren'] = (dfb['o'] - dfb['o_bren']).apply(abs)
print('Error cuadratico medio biseccion: ', np.square(dfb['o'] - dfb['o_bis']).mean())
print('Error absoluto medio biseccion: ', dfb['diff_bis'].mean())
aux = 100*(np.abs(dfb['o'] - dfb['o_bis']) / dfb['o']).mean() 
print('Error absoluto porcentual medio biseccion', aux)
print('r2 Biseccion', r2_score(dfb['o'], dfb['o_bis']))

print('\n\n')

print('Error cuadratico medio Brent: ', np.square(dfb['o'] - dfb['o_bren']).mean())
print('Error absoluto medio Brent: ', dfb['diff_bren'].mean())
aux = 100*(np.abs(dfb['o'] - dfb['o_bren']) / dfb['o']).mean() 
print('Error absoluto porcentual medio Brent', aux)
print('r2 Brent', r2_score(dfb['o'], dfb['o_bren']))

Error cuadratico medio biseccion:  9.265692032006615e-09
Error absoluto medio biseccion:  2.11098356813282e-06
Error absoluto porcentual medio biseccion 0.0029601293365861133
r2 Biseccion 0.9999998814466189



Error cuadratico medio Brent:  2.718659948523773e-06
Error absoluto medio Brent:  5.9285333962340034e-05
Error absoluto porcentual medio Brent 0.2745302521588844
r2 Brent 0.999965215082936


In [11]:
#Ordeno el dataframe segun el error entre la volatilidad estimada
# y la volatilidad implícita del método de bisección
orda = dfb.sort_values('diff_bis', ignore_index=True, ascending= False)

In [12]:
#aplico la función que busca la raiz sobre la volatilidad generada
#mediante el método de bisección
fs = []
auxi = orda[['c','S', 'k', 'r', 'o_bis', 'T']]
for i in range(len(auxi)):
    fs.append(raiz(*auxi.iloc[i]))

orda['f(o_bis)'] = fs


In [13]:
orda.head()

,c,S,k,r,T,o,o_bis,o_bren,diff_bis,diff_bren,f(o_bis)
0,49.088954,99.235069,53.355759,0.084518,0.734059,0.095865,0.103779,0.102826,0.007915,0.006961,0.0
1,27.540779,87.878087,62.188114,0.027988,1.079502,0.047555,0.044563,0.044592,0.002992,0.002964,0.0
2,28.161195,96.701943,69.353868,0.034968,0.337262,0.076701,0.079609,0.078679,0.002909,0.001978,0.0
3,26.938256,76.948774,52.467325,0.053914,0.889515,0.057501,0.060273,0.060376,0.002772,0.002876,0.0
4,41.280944,81.268902,41.807108,0.042913,1.036700,0.091623,0.090244,0.088067,0.001379,0.003556,0.0


Observar q en muchos casos la diferencia entre la volatilidad implícita y la volatilidad estimada es grande, eso es porque en ciertos casos la volatilidad tiene poco impacto, osea un "vega bajo", y por el problema de precisión la función que calcula la raíz retorna 0.

In [14]:

#vega en funcion de ratio. La función retorna vega/K.
def vega(S, K, r, o, T):
    
    d1 = (np.log(S/K) + (r + 0.5 * o ** 2) * T) / (o * np.sqrt(T))
    
    vega = np.sqrt(T) * S * math.exp(-d1**2/2) / np.sqrt(2*math.pi)
    
    return vega

In [15]:
SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

fi_1 = '\u03A61'.translate(SUB)+'(o)'
fi_2 = '\u03A62'.translate(SUB)+'(o)'
fi_1_b = '\u03A61'.translate(SUB)+'(o_bis)'
fi_2_b = '\u03A62'.translate(SUB)+'(o_bis)'
fi_1_01 = '\u03A61'.translate(SUB)+'(0.01)'
fi_2_01 = '\u03A62'.translate(SUB)+'(0.01)'

fi_1, fi_2, fi_1_01, fi_2_01

('Φ₁(o)', 'Φ₂(o)', 'Φ₁(0.01)', 'Φ₂(0.01)')

In [16]:
def my_vega(elem):
    return vega(elem['S'],elem['k'], elem['r'], elem['o'], elem['T'])

d_1 = d1(*orda[['S','k','r','o','T']].iloc[0])
#d_2 = d_1 - orda.iloc[0]['o']*math.sqrt(orda.iloc[0]['T']) 
d_2 = d_1 - 0.01*math.sqrt(orda.iloc[0]['T']) 
norm1 = st.norm.cdf(d_1)
d_1_p =d1(*orda[['S','k','r']].iloc[0],0.01,orda.iloc[0]['T'])
st.norm.cdf(d_1), st.norm.cdf(d_2)

def my_norm_1(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], elem['o'], elem['T'])
    return st.norm.cdf(d_1)

def my_norm_2(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], elem['o'], elem['T'])
    return st.norm.cdf(d_1 - elem['o']*math.sqrt(elem['T']))

def my_norm_1_b(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], elem['o_bis'], elem['T'])
    return st.norm.cdf(d_1)

def my_norm_2_b(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], elem['o_bis'], elem['T'])
    return st.norm.cdf(d_1 - elem['o']*math.sqrt(elem['T']))

def my_norm_1_001(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], 0.01, elem['T'])
    return st.norm.cdf(d_1)

def my_norm_2_001(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], 0.01, elem['T'])
    return st.norm.cdf(d_1 - 0.01*math.sqrt(elem['T']))


orda['vega'] = orda.apply(my_vega, axis=1)
orda[fi_1] = orda.apply(my_norm_1, axis=1)
orda[fi_2] = orda.apply(my_norm_2, axis=1)
orda[fi_1_01] = orda.apply(my_norm_1_001, axis=1)
orda[fi_2_01] = orda.apply(my_norm_2_001, axis=1)
orda[fi_1_b] = orda.apply(my_norm_1_b, axis=1)
orda[fi_2_b] = orda.apply(my_norm_2_b, axis=1)


In [17]:
orda.head(50)

,c,S,k,r,T,o,o_bis,o_bren,diff_bis,diff_bren,f(o_bis),vega,Φ₁(o),Φ₂(o),Φ₁(0.01),Φ₂(0.01),Φ₁(o_bis),Φ₂(o_bis)
0,49.088954,99.235069,53.355759,0.084518,0.734059,0.095865,0.103779,0.102826,7.914706e-03,6.961251e-03,0.0,2.431290e-14,1.0,1.0,1.0,1.0,1.0,1.0
1,27.540779,87.878087,62.188114,0.027988,1.079502,0.047555,0.044563,0.044592,2.992464e-03,2.963724e-03,0.0,8.029539e-12,1.0,1.0,1.0,1.0,1.0,1.0
2,28.161195,96.701943,69.353868,0.034968,0.337262,0.076701,0.079609,0.078679,2.908536e-03,1.978404e-03,0.0,2.037144e-12,1.0,1.0,1.0,1.0,1.0,1.0
3,26.938256,76.948774,52.467325,0.053914,0.889515,0.057501,0.060273,0.060376,2.772485e-03,2.875507e-03,0.0,4.556096e-13,1.0,1.0,1.0,1.0,1.0,1.0
4,41.280944,81.268902,41.807108,0.042913,1.036700,0.091623,0.090244,0.088067,1.378604e-03,3.555914e-03,0.0,6.532176e-12,1.0,1.0,1.0,1.0,1.0,1.0
5,20.389259,61.124476,42.965539,0.051076,1.043641,0.051492,0.052539,0.052615,1.047106e-03,1.122947e-03,0.0,2.422918e-12,1.0,1.0,1.0,1.0,1.0,1.0
6,23.553129,58.305847,38.556694,0.097293,1.067615,0.067627,0.067283,0.066991,3.440482e-04,6.354684e-04,0.0,2.293169e-11,1.0,1.0,1.0,1.0,1.0,1.0
7,31.142404,62.263444,31.957304,0.072702,0.364732,0.157000,0.156711,0.156766,2.890725e-04,2.349626e-04,0.0,2.564326e-11,1.0,1.0,1.0,1.0,1.0,1.0
8,23.522932,90.384156,67.268484,0.022837,0.265910,0.080598,0.080334,0.080323,2.630317e-04,2.741646e-04,0.0,6.027168e-11,1.0,1.0,1.0,1.0,1.0,1.0
9,23.128934,71.206369,50.459553,0.063074,0.766708,0.059865,0.060032,0.059880,1.667675e-04,1.492264e-05,0.0,1.314932e-11,1.0,1.0,1.0,1.0,1.0,1.0


In [18]:
#Los que tienen mas error cumplen esa condicion
#Obsevar en la celda siguiente a esta que hay vetas mucho menor
# a este ejemplo, y sin embargo tiene un error mucho mas grande
print(fi_1 + ': {}'.format(orda.iloc[0][fi_1]))
print(fi_2 + ': {}'.format(orda.iloc[0][fi_2]))
print(fi_1_b + ': {}'.format(orda.iloc[0][fi_1_b]))
print(fi_2_b + ': {}'.format(orda.iloc[0][fi_2_b]))

print(fi_1_01 + ': {}'.format(orda.iloc[0][fi_1_01]))
print(fi_2_01 + ': {}'.format(orda.iloc[0][fi_2_01]))
print('f(o_bis): ', orda['f(o_bis)'].iloc[0])

Φ₁(o): 1.0
Φ₂(o): 0.9999999999999999
Φ₁(o_bis): 0.9999999999999942
Φ₂(o_bis): 0.999999999999989
Φ₁(0.01): 1.0
Φ₂(0.01): 1.0
f(o_bis):  0.0


In [19]:

with open('dataFrame/orda.pickle', 'wb') as handle:
    pickle.dump(orda, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
with open('dataFrame/orda.pickle', 'rb') as handle:
    ordy = pickle.load(handle)
ordy.head()

,c,S,k,r,T,o,o_bis,o_bren,diff_bis,diff_bren,f(o_bis),vega,Φ₁(o),Φ₂(o),Φ₁(0.01),Φ₂(0.01),Φ₁(o_bis),Φ₂(o_bis)
0,49.088954,99.235069,53.355759,0.084518,0.734059,0.095865,0.103779,0.102826,0.007915,0.006961,0.0,2.431290e-14,1.0,1.0,1.0,1.0,1.0,1.0
1,27.540779,87.878087,62.188114,0.027988,1.079502,0.047555,0.044563,0.044592,0.002992,0.002964,0.0,8.029539e-12,1.0,1.0,1.0,1.0,1.0,1.0
2,28.161195,96.701943,69.353868,0.034968,0.337262,0.076701,0.079609,0.078679,0.002909,0.001978,0.0,2.037144e-12,1.0,1.0,1.0,1.0,1.0,1.0
3,26.938256,76.948774,52.467325,0.053914,0.889515,0.057501,0.060273,0.060376,0.002772,0.002876,0.0,4.556096e-13,1.0,1.0,1.0,1.0,1.0,1.0
4,41.280944,81.268902,41.807108,0.042913,1.036700,0.091623,0.090244,0.088067,0.001379,0.003556,0.0,6.532176e-12,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
#Observar q los casos con mas error son ITM
#Fitro por ratio < 1 OTM
#NOTAR QUE HAY CASOS CON VEGA EXTREMADAMENTE CHICO PERO SU ERROR NO ES GRANDE
orda[orda['S']/orda['k'] < 1].head(50)


In [ ]:
#porcentaje de elementos en que no se encontro la raiz
# mediante el método de bisección con tolerancia 2**-56
100*len(orda[orda['f(o_bis)'] != 0])/len(orda)
#quiere decir q no termina, si no que corta a las 100 iteraciones


In [ ]:
ordas = orda.sort_values('vega', ignore_index=True)

In [ ]:
auxi = ordas[(ordas['o'] < 0.52) & (ordas['o'] > 0.48)]
vega_chico = auxi.iloc[0]
vega_grande = auxi.iloc[-1]
vega_mediano = auxi.iloc[len(auxi)//2]
vega_new = auxi.iloc[len(auxi)//4]
print('Vega Chico: ', vega_chico['vega'])
print('Vega Mediano: ', vega_mediano['vega'])
print('Vega Grande: ', vega_grande['vega'])


In [ ]:
def my_call_price(o, elem=vega_chico):
    return call_price(elem['S'], elem['k'], elem['r'], o, elem['T'])

sdvc = np.vectorize(my_call_price)

f = lambda x: my_call_price(x, elem=vega_grande)
sdvg = np.vectorize(f)

m = lambda x: my_call_price(x, elem=vega_mediano)
sdvm = np.vectorize(m)

In [ ]:
fig, ax, = plt.subplots(figsize=(15,10))
volatilidad = np.linspace(0.001, 2, 1000)
#ax.plot(history_clr.history['loss'], 'b', label = 'implied volatility')
my_round = lambda x: round(x, 4)
ax.plot(volatilidad, sdvc(volatilidad),'m',
    label='Vega Chico S: {}, K: {}, r: {}, T: {}, vega(o=0.5): {}'.format(*vega_chico[['S','k', 'r', 'T', 'vega']].apply(my_round)))

######################################################
ax.plot(volatilidad, sdvm(volatilidad),'k',
    label='Vega Mediano S: {}, K: {}, r: {}, T: {}, vega(o=0.5): {}'.format(*vega_mediano[['S','k', 'r', 'T', 'vega']].apply(my_round)))
######################################################
ax.plot(volatilidad, sdvg(volatilidad),'b',
    label='Vega Grande S: {}, K: {}, r: {}, T: {}, vega(o=0.5): {}'.format(*vega_grande[['S','k', 'r', 'T', 'vega']].apply(my_round)))
#########################################################

ax.legend()
plt.legend(fontsize=14)
plt.title('Comparacion vega vs volatilidad')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("volatilidad", fontsize=18)
plt.ylabel("precio call", fontsize=18)
plt.grid()
plt.savefig('volatilidad_vega',dpi=300, bbox_inches='tight')
plt.show()